In [1]:
import networkx as nx
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.sparse.linalg import eigs
from collections import defaultdict
import pandas as pd
import os
import time

In [2]:
class Walktrap:
    def __init__(self, G, steps=4):
        self.G = G
        self.steps = steps
        self.n = len(G)
        self.P = self._get_transition_matrix()
        self.P_t = self._get_random_walk_matrix()
        
    def _get_transition_matrix(self):
        A = nx.adjacency_matrix(self.G).toarray()
        degrees = np.sum(A, axis=1)
        degrees[degrees == 0] = 1  # Avoid division by zero
        D_inv = np.diag(1.0 / degrees)
        return np.dot(D_inv, A)
    
    def _get_random_walk_matrix(self):
        return np.linalg.matrix_power(self.P, self.steps)
    
    def _calculate_distance(self, community1, community2):
        prob1 = np.mean([self.P_t[i] for i in community1], axis=0)
        prob2 = np.mean([self.P_t[i] for i in community2], axis=0)
        degrees = np.array([self.G.degree(i) for i in range(self.n)])
        degrees[degrees == 0] = 1  # Avoid division by zero
        diff = prob1 - prob2
        weighted_diff = diff * diff / degrees
        return np.sqrt(np.sum(weighted_diff))
    
    def _prepare_distance_matrix(self):
        dist_matrix = np.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(i + 1, self.n):
                dist = self._calculate_distance({i}, {j})
                dist_matrix[i, j] = dist_matrix[j, i] = dist
        return dist_matrix
    
    def detect_communities(self, n_communities=None):
        dist_matrix = self._prepare_distance_matrix()
        condensed_dist = dist_matrix[np.triu_indices(self.n, k=1)]
        Z = linkage(condensed_dist, method='ward')
        
        if n_communities is None:
            n_communities = self._find_optimal_communities(Z)
            
        labels = fcluster(Z, n_communities, criterion='maxclust') - 1  # Zero-based indexing
        
        communities = defaultdict(list)
        for node, label in enumerate(labels):
            if node < self.n:  # Only include actual nodes
                communities[label].append(node)
                
        return dict(communities)
    
    def _find_optimal_communities(self, Z):
        best_modularity = -1
        best_n = 1
        
        for n in range(2, min(11, self.n)):
            labels = fcluster(Z, n, criterion='maxclust')
            modularity = self._calculate_modularity(labels)
            
            if modularity > best_modularity:
                best_modularity = modularity
                best_n = n
                
        return best_n
    
    def _calculate_modularity(self, labels):
        m = self.G.number_of_edges()
        if m == 0:
            return 0
            
        Q = 0
        for i in range(self.n):
            for j in range(self.n):
                if labels[i] == labels[j]:
                    A_ij = 1 if self.G.has_edge(i, j) else 0
                    k_i = self.G.degree(i)
                    k_j = self.G.degree(j)
                    Q += A_ij - (k_i * k_j) / (2 * m)
                    
        return Q / (2 * m)

In [3]:
def load_adjacency_matrix(csv_path):
    adj_matrix = pd.read_csv(csv_path, header=None).values
    G = nx.from_numpy_array(adj_matrix)
    return G

In [4]:
def save_results(communities, output_path):
    result_dict = {}
    with open(output_path, 'w') as f:
        for i, community in enumerate(communities.values()):
            for comm in community:
                result_dict[comm] = i

        result_dict = dict(sorted(result_dict.items()))
        for key, value in result_dict.items():
            f.write(str(key + 1) + ', ' + str(value + 1) + '\n')

## Unknown communities

In [5]:
data_folder = 'data_unc'
files = os.listdir(data_folder)
print(files)

for file in files:
    G = load_adjacency_matrix(data_folder + '/' + file)
    
    start_time = time.time()
    walktrap = Walktrap(G, steps=10)
    communities = walktrap.detect_communities()
    print(file, time.time() - start_time)
    
    save_results(communities, 'results/' + file)

['D1-UNC.csv', 'D2-UNC.csv', 'D3-UNC.csv']
D1-UNC.csv 1.0126221179962158
D2-UNC.csv 1.5530200004577637
D3-UNC.csv 0.3162250518798828


## Known communities

In [6]:
data_folder = 'data'
files = os.listdir(data_folder)
print(files)

for file in files:

    G = load_adjacency_matrix(data_folder + '/' + file)
    
    start_time = time.time()
    walktrap = Walktrap(G, steps=10)
    communities = walktrap.detect_communities(int(file[file.find('=') + 1:file.find('.')]))
    print(file, time.time() - start_time)
    
    save_results(communities, 'results/' + file)

['D1-K=2.csv', 'D2-K=7.csv', 'D3-K=12.csv']
D1-K=2.csv 0.04101157188415527
D2-K=7.csv 0.17496609687805176
D3-K=12.csv 0.6961531639099121
